# Basic Interface

First we chnage to our dev directory so we can use the python class

In [1]:
import os
os.chdir("/Users/drbh2/Desktop/nucypher-ipfs")

now we load in the **ncipfs** class and instantiate a client

In [14]:
from ncipfs import main

client = main.ncipfs()

now we pass the endpoints for IPFS and the Ursula network - here we connect to a local federated network, and infuras IPFS public gateway

We also load in an exisiting identity (or save a new one to the directory we specify. We'll just add this dir to our current location)

In [16]:
client.connect(nucypher_network="localhost:11500",
          ipfs_api_gateway="https://ipfs.infura.io:5001")

client.load_user("alice_ipfs_user/")

Automatic Mode A Public Gateway will be used as a fallback


(Alice)⇀LightPink Thunderstorm Gray Sagittarius↽ (0x7060DB53dF834CD58a74138C817c4eebb0eA4963)

# Securely Adding Data to IPFS

Next we specify the name of the file we want to add to IPFS and its string contents. This label helps the Ursula network re-encrypt the data correctly but will not be the loaction of the file on IPFS. This will be a hash that starts with `Qm`...  Later work will help map these values together

In [20]:
my_label = b'ncipfs_is_awesome.txt'
my_contents = "arbitrary data that is stored on IPFS"

In [21]:
pubkey = client.make_key_from_label(my_label)
cid = client.add_contents(pubkey, my_contents)

The policy public key for label 'ncipfs_is_awesome.txt' is 029412cf5595e3b7662204e2ebe036b74776dafd43edc883e401518b5d356506b8
🚀 ADDING TO IPFS D-STORAGE NETWORK 🚀
File Address:	QmV6SmRwYkStx1sUUia2upUkoCw83zGb98MHz6WnbD3jwV


Yay! we now have our encrypted data stored in IPFS at located at the CID above.

# Permissioning Access to Data

In this demo we will create the private and public keys for the recipent, but these will likely be stored values in the future.

We only need the public values to 

In [6]:
from umbral.keys import UmbralPrivateKey, UmbralPublicKey

enc_privkey = UmbralPrivateKey.gen_key()
sig_privkey = UmbralPrivateKey.gen_key()

enc_pubkey = enc_privkey.get_pubkey()
sig_pubkey = sig_privkey.get_pubkey()

no we need to specify the label of the file

In [7]:
label = b'ncipfs_is_awesome.txt'
m, n = 2, 3

In [8]:
policy = client.create_access_policy(
    enc_pubkey=enc_pubkey, 
    sig_pubkey=sig_pubkey, 
    label=label, 
    m=m, 
    n=n   
)
print(policy)

Creating access policy for the Doctor...


In [10]:
doctor = client.load_recipent(enc_privkey, sig_privkey)

In [11]:
doctor

(Bob)⇀OliveDrab Swords Chartreuse Queen↽ (0xEec440D0e7ebd35a8002aEd71f4e50eE627aCCA4)

# Accessing the Data 

Now we want to fetch, ask for re encryption and then decrypt the data.

In [12]:
message = client.get_file_and_decrypt(doctor, policy, cid)

The Doctor joins policy for label 'ncipfs_is_awesome.txt'


In [13]:
message

'arbitrary data that is stored on IPFS   Retrieval time:    97.65 ms'